# Set Path & Model 

In [1]:
# model_path = './models/RegNet_0.001_Lamb_CosineAnnealing_example.pth' # *
# model_path = './models/RegNet_0.001_adam_CosineAnnealing_example.pth'
# model_path = './models/RegNet_1e-05_adam_CosineAnnealing_example.pth'
# model_path = './models/RegNet_1e-05_Lamb_CosineAnnealing_example.pth'
model_path = './models/RegNet_1e-05_rmsprop_CosineAnnealing_example.pth' # *
# model_path = './models/RegNet_1e-05_nadam_CosineAnnealing_example.pth'
# model_path = './models/ResNet50_0.001_adam_CosineAnnealing_example.pth'
# model_path = './models/ResNet50_0.001_Lamb_CosineAnnealing_example.pth'
# model_path = './models/ResNet50_0.001_rmsprop_CosineAnnealing_example.pth'
# model_path = './models/ResNet50_1e-05_Lamb_CosineAnnealing_example.pth'
# model_path = './models/ResNet50_1e-05_adam_CosineAnnealing_example.pth'
# model_path = './models/EfficientNetb4_0.001_adam_CosineAnnealing_example.pth'
# model_path = './models/EfficientNetb4_0.001_Lamb_CosineAnnealing_example.pth'
# model_path = './models/EfficientNetb4_1e-05_rmsprop_CosineAnnealing_example.pth'

In [2]:
# path = 'D:/jupyter_home/3_project/test_img/test_all/' 
path = 'D:/jupyter_home/3_project/test_img/test_one/' 

In [3]:
# used_model = 'resnet'
# used_model = 'efficientnet'
used_model = 'regnet'

# Library

In [4]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from os.path import join as opj
from glob import glob

import timm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
import albumentations as A

import warnings
warnings.filterwarnings('ignore')

# Dataset & Loader

In [5]:
class Test_dataset(Dataset):
    def __init__(self, path, transform=None):
        total_images_path = glob(path + '*.jpg')
        file_names = []
        for i in range(len(total_images_path)):
            file_names.append(os.path.basename(total_images_path[i]))
            file_names.sort()
        file_names = np.array(file_names)

        self.test_file_name = file_names
        self.transform = transform

        print(f'Test Dataset size : {len(self.test_file_name)}')
        print(self.test_file_name)

    def __getitem__(self, idx): # test 경로에 있는 png 이미지 읽어서 float32로 변환
        image = cv2.imread(opj(path, self.test_file_name[idx])).astype(np.float32)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) / 255.0  # BGR=>RGB 변환

        if self.transform is not None:
            image = self.transform(torch.from_numpy(image.transpose(2,0,1)))

        return image

    def __len__(self):
        return len(self.test_file_name)

In [6]:
def get_test_augmentation(img_size):
    transform = transforms.Compose([
                transforms.Resize((img_size, img_size)),
                transforms.Normalize(mean=(0.744859, 0.735139, 0.711357), std=(0.100712, 0.120692, 0.167998)),  
                ])
    
    return transform

In [7]:
test_transform = get_test_augmentation(img_size=256)
test_dataset = Test_dataset(path, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False, num_workers=0, collate_fn=None)

Test Dataset size : 1
['55702_1.jpg']


# Model

In [8]:
CFG = {
    'IMG_SIZE':256,
    'EPOCHS':50,
    'PATIENCE':10,
    'class':14
}

In [9]:
class ResNet50(torch.nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()
        model = models.resnet50(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.fc1 = nn.Linear(2048, 1000)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1000,CFG['class'])

    def forward(self, x):
        x = self.feature_extract(x)
        # x = x.mean(dim=(-2, -1))
        # (batch, 2048, 4, 4)
#         x = torch.squeeze(x)
        x = torch.flatten(x, 1)

        x = self.relu(self.fc1(x))
        out = self.fc2(x)
        
        return out

In [10]:
class EfficientNetb4(torch.nn.Module):
    def __init__(self):
        super(EfficientNetb4, self).__init__()
        model = models.efficientnet_b4(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.fc1 = nn.Linear(1792, 1000)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1000, CFG['class'])
        
    def forward(self, x):
        x = self.feature_extract(x)
        # (batch, 1792, 1, 1)
#         x = torch.squeeze(x)
        x = torch.flatten(x, 1)

        x = self.relu(self.fc1(x))
        out = self.fc2(x)
        
        return out

In [11]:
class RegNet(torch.nn.Module):
    def __init__(self):
        super(RegNet, self).__init__()
        model = models.regnet_y_16gf(pretrained=True)
        modules = list(model.children())[:-1]
        self.feature_extract = nn.Sequential(*modules)
        self.fc1 = nn.Linear(3024, 1000)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1000, CFG['class'])
        
    def forward(self, x):
        x = self.feature_extract(x)
        # (batch, 3024, 1, 1)
        
#         x = torch.squeeze(x)
        x = torch.flatten(x, 1)
        
        x = self.relu(self.fc1(x))
        out = self.fc2(x)
        
        return out

# Prediction

In [12]:
def predict(encoder_name, test_loader, device, model_path):
    if encoder_name == 'resnet':
        model = ResNet50().to(device)
    elif encoder_name == 'efficientnet':
        model = EfficientNetb4().to(device)
    elif encoder_name == 'regnet':
        model = RegNet().to(device)
        
    model.load_state_dict(torch.load(model_path))
    model.eval()
    preds_list = []
    with torch.no_grad():
        for images in tqdm(test_loader):
            images = torch.as_tensor(images, device=device, dtype=torch.float32)
            preds = model(images)
            preds = torch.softmax(preds, dim=1)
            preds_list.extend(preds.cpu().tolist())

    return np.array(preds_list)

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
predict_arr = predict(used_model, test_loader, device, model_path)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.40s/it]


In [15]:
predict_arr

array([[1.53626781e-04, 1.74887409e-05, 3.37045931e-04, 1.27747902e-04,
        2.58083828e-03, 3.45359440e-04, 5.63827343e-03, 7.01616409e-06,
        2.11710273e-03, 1.11962894e-04, 5.38883323e-04, 1.89988015e-04,
        1.13925757e-03, 9.86695409e-01]])

In [16]:
predict_arr.argmax(axis=1)

array([13], dtype=int64)

In [17]:
labels = ['25222_대만)망고케익184g', '25228_대만)파인애플케익184G', '35211_매일유업)데르뜨130G', '35584_매일데르뜨파인애플90G', '35585_매일데르뜨감귤90G', '45030_돌황도666G', '45657_씨제이)쁘티첼(요거젤리복숭아)', '45658_씨제이)쁘티첼(요거젤리밀감)', '45659_씨제이)쁘티첼(요거젤리딸기)', '45660_씨제이)쁘티첼(요거젤리화이트코코)', '45661_씨제이)쁘티첼(요거젤리블루베리)', '55034_돌트로피칼666G', '55701_쁘띠첼요거젤리밀감', '55702_쁘띠첼요거젤리복숭아']

In [18]:
labels = {string : i for i, string in enumerate(labels)}
label_decoder = {val:key for key, val in labels.items()}

In [19]:
pred_list = []

for i in range(len(test_dataset.test_file_name)):
    prediction = label_decoder[predict_arr.argmax(axis=1)[i]]
    print(prediction)
    pred_list.append(prediction)

55702_쁘띠첼요거젤리복숭아


In [20]:
# pred_list

# Accuracy

In [21]:
# from sklearn import preprocessing
# import torch.nn.functional as F
# import albumentations as A
# import albumentations.pytorch

In [22]:
# def get_valid_data(data_dir):
#     img_valid_list = []
#     label_valid_list = []
    
#     image_path = os.path.join(data_dir, 'dessert')
    
#     for product_name in os.listdir(image_path):
#         product_path = os.path.join(image_path, product_name)
#         if os.path.isdir(product_path):
#             # get image path
#             img_valid_list.extend(glob(os.path.join(product_path, '*.jpg')))
#             img_valid_list.extend(glob(os.path.join(product_path, '*.png')))
#             label = list(product_name[:5])
            
#             # get label
#             label_valid_list.append(''.join(label))
                
#     return img_valid_list, label_valid_list

In [23]:
# def valid_data_blanced(img, label):
#     x = []
#     y = []
    
#     for i in range(CFG['class']):
#         _img = img[(i * 15): ((i + 1) * 15)]
#         _label = label[i]
        
#         for img_product in _img:
#             x.append(img_product)
#             y.append(_label)
            
#     return x, y

In [24]:
# img_valid_list, label_valid_list = get_valid_data('./Data/product_image/Validation/')
# x_valid, y_valid = valid_data_blanced(img_valid_list, label_valid_list)
# len(label_valid_list)

In [25]:
# le2 = preprocessing.LabelEncoder()
# targets_y = le2.fit_transform(y_valid)
# targets_y = torch.as_tensor(targets_y)
# one_hot_valid_y = F.one_hot(targets_y)
# one_hot_valid_y.shape

In [26]:
# class AlbumentationsCustomDataset(Dataset):
#     def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
#         self.transforms = transforms
#         self.train_mode = train_mode
#         self.img_path_list = img_path_list
#         self.label_list = label_list

#     def __getitem__(self, index):
#         img_path = self.img_path_list[index]
#         # Get image data
#         image = cv2.imread(img_path)
        
#         # By default OpenCV uses BGR color space for color images,
#         # so we need to convert the image to RGB color space.
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

#         if self.train_mode:
# #             image = image.astype(np.int16)
#             augmented = self.transforms(image=image)
#             image = augmented['image']
#             label = self.label_list[index]
#             return image, label
#         else:
#             image = self.transforms(image)
#             label = self.label_list[index]
#             return image, label
    
#     def __len__(self):
#         return len(self.img_path_list)

In [27]:
# A_test_transform = albumentations.Compose([
#                                     A.Resize(256, 256),
#                                     A.Normalize(mean=(0.744859, 0.735139, 0.711357), std=(0.100712, 0.120692, 0.167998)),  
# #                                     A.pytorch.transforms.ToTensor(),
#                                     A.pytorch.transforms.ToTensorV2(transpose_mask=True),
#                                 ])

In [28]:
# A_vali_dataset = AlbumentationsCustomDataset(x_valid, one_hot_valid_y, train_mode=True, transforms=A_test_transform)
# A_vali_loader = DataLoader(A_vali_dataset, batch_size = 5, shuffle=False, num_workers=0, collate_fn=None)

In [29]:
# def predict_for_acc(model, test_loader, device):
#     model.eval()
#     model_pred = []
#     with torch.no_grad():
#         for img, label in tqdm(iter(test_loader)):
#             img = img.float().to(device)
            
#             pred_logit = model(img)
#             pred_logit = pred_logit.squeeze().detach().cpu()
            
#             model_pred.extend(pred_logit.tolist())
#     return model_pred

In [30]:
# checkpoint = torch.load(model_path)

In [31]:
# if used_model == 'resnet':
#     model = ResNet50().to(device)
# elif used_model == 'efficientnet':
#     model = EfficientNetb4().to(device)
# elif used_model == 'regnet':
#     model = RegNet().to(device)

In [32]:
# model.load_state_dict(checkpoint)

# preds = predict_for_acc(model, A_vali_loader, device)

In [33]:
# pred_labels = np.argmax(preds, axis=1)
# true_labels = one_hot_valid_y.argmax(-1)

In [34]:
# from sklearn.metrics import accuracy_score

# accuracy_score(true_labels, pred_labels)